## Basic Message Routing from YAML Spec (Format Based)

This notebook demonstrates simple routing, picking up messages from one topic and writing it to different topic based on the message format.

Simple routing allows filtering using:
- origin_sender
- origin_topic
- origin_message_format

In this case, we are routing based on origin message format.

[ProbeAgent] --_MessageType1_-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_1**)

[ProbeAgent] --_MessageType2_-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_2**)

In [1]:
# Define different message types

from pydantic import BaseModel

class MessageType1(BaseModel):
    content: str

class MessageType2(BaseModel):
    content: str
    additional_info: str

In [2]:
from rustic_ai.core.guild.builders import GuildBuilder

guild_builder = GuildBuilder.from_yaml_file("./002_basic_message_routing.yaml")
guild = guild_builder.launch("myorg")

In [3]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent
from rustic_ai.core.guild.builders import AgentBuilder
from rustic_ai.core.guild.dsl import GuildTopics

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test the routing of messages.")
    .add_additional_topic("topic_1")
    .add_additional_topic("topic_2")
    .build()
)

guild._add_local_agent(probe_agent)

In [4]:
probe_agent.publish_with_guild_route(
    payload=MessageType1(content="Hello from MessageType1!"),
    topic=guild_default_topic,
)

In [5]:
probe_agent.print_message_history()

(default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (topic_1)


In [6]:
probe_agent.publish_with_guild_route(
    payload=MessageType2(content="Hello from MessageType2!", additional_info="Some extra info"),
    topic=guild_default_topic,
)

In [7]:
probe_agent.print_message_history()

(default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (topic_2)
